In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Ce projet a pour but de créer un modèle prédisant si un futur client d'une compagnie va y rester fidèle.
## On se donne pour objectif une précision de 70% et un recall de 60%.

In [ ]:
#Import de modules
import matplotlib.pyplot as plt
import seaborn as sns

## Chargement des données

In [ ]:
df = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')

In [ ]:
df.head()

In [ ]:
df.shape

## Description du Dataset

In [ ]:
df.describe()

In [ ]:
df.info()

L'absence de valeurs nulles va grandement nous faciliter la tâche.

In [ ]:
#Répartition des types de données
df.dtypes.value_counts().plot.pie()
plt.show()

## Examen de la colonne Target

On sait que "Exited" = 0 signifie qu'un client reste au côté de l'entreprise et donc que "Exited" = 1 signifie qu'un client quitte l'entreprise.

In [ ]:
df['Exited'].value_counts()

Il y a plus de clients restant dans l'entreprise que de clients la quittant.

In [ ]:
#Exprimé en probabilité
df['Exited'].value_counts(normalize=True)

In [ ]:
df['Exited'].value_counts(normalize=True).plot.pie()
plt.show()

Les scores ne sont pas équilibrés, il faudra utiliser une mesure d'évalutation comme : le score F1, la sensibilité, la précision.

## Analyse des variables quantitatives

In [ ]:
#Histogrammes des float
for col in df.select_dtypes('float') :
    plt.figure()
    sns.histplot(df[col])
    plt.show()

In [ ]:
int_continues = [col for col in df.select_dtypes('int')][2:4]

In [ ]:
#Histogrammes des int
for col in int_continues :
    plt.figure()
    sns.histplot(df[col])
    plt.show()

On remarque quelques outliers pour les variables "Balance" et "CreditScore".

## Analyse de nos variables qualitatives

In [ ]:
#Liste des différentes valeurs que peuvent prendre les variables qualitatives
for col in df.select_dtypes('object') :
    print(f'{col :<10}, {df[col].unique()}')

In [ ]:
liste_object = [col for col in df.select_dtypes('object')]
object_clear = liste_object[1:]

for col in object_clear :
    plt.figure()
    df[col].value_counts().plot.pie()

Répartition sur 3 pays avec 50% de clients Français.

In [ ]:
int_cat = [col for col in df.select_dtypes('int')][4:-1]

In [ ]:
for col in int_cat :
    print(f'{col :<15}, {df[col].unique()}')

In [ ]:
for col in int_cat :
    plt.figure()
    df[col].value_counts().plot.pie()

# Analyse de la relation Features / Target

## Features qualitatives vs target

In [ ]:
#Répartition des pays et du genre sur la target
for col in object_clear :
    plt.figure()
    sns.heatmap(pd.crosstab(df['Exited'], df[col]), annot = True, fmt = 'd')

In [ ]:
for col in object_clear :
    sns.catplot(data=df, x="Exited", hue=col, kind = 'count')
    plt.show()

In [ ]:
for col in int_cat :
    sns.catplot(data=df, x="Exited", hue=col, kind = 'count')
    plt.show()

Les membres possédant un seul produit, ou étant inactifs sont plus à même de quitter l'entreprise.

## Features quantitatives vs target

#### Création de sous-ensemble Exited / non-Exited

In [ ]:
exited_df = df[df['Exited'] == 1]
non_exited_df = df[df['Exited'] == 0]

In [ ]:
continuous_columns = df[['CreditScore', 'Age', 'Balance', 'EstimatedSalary']].columns

In [ ]:
for col in continuous_columns :
    sns.histplot(data= exited_df[col], label = 'Exited', kde = True, color = 'red')
    sns.histplot(data= non_exited_df[col], label = 'Non-Exited', kde = True, color = 'orange')
    plt.legend()
    plt.show()

Les facteurs décisifs semblent être l'âge et le score de crédit.

# Développement de modèles

## Features Engineering

On a remarqué que les variables "CreditScore" et "Balance" présentent des outliers. Nous allons les supprimer.

In [ ]:
#On crée une copie propre de notre dataframe
dfml = df.copy()

In [ ]:
#Fonction supprimant les outliers et retirant quelques colonnes inutiles
def features_engineering(dfml) : 
    dfml = dfml[dfml['CreditScore'] < 850]
    dfml = dfml[dfml['Balance'] > 0]
    dfml.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)
    return dfml

In [ ]:
dfml = features_engineering(dfml)

In [ ]:
dfml

Notre dataframe a été correctement formaté.

## Création du train set et du test set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#On crée un test_set de 20% de la taille de notre dataframe initial
train_set, test_set = train_test_split(dfml, test_size = 0.2)

In [ ]:
train_set['Exited'].value_counts()

In [ ]:
test_set['Exited'].value_counts()

Les deux classes de notre modèle sont correctements réparties sur le train et le test set

In [ ]:
#Séparation des features et des targets
X_train = train_set.drop('Exited', axis = 1)
y_train = train_set['Exited']
X_test = test_set.drop('Exited', axis = 1)
y_test = test_set['Exited']

## Pre-processing

In [ ]:
#importation des différents outils sklearn
from sklearn.preprocessing import RobustScaler, StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
#On sépare les features quantitatives et qualitatives
numerical_features = make_column_selector(dtype_include = np.number)
categorical_features = make_column_selector(dtype_exclude = np.number)

In [ ]:
#On normalise les features quantitatives, on encode les features qualitatives
numerical_pipeline = make_pipeline(RobustScaler(), SelectKBest(f_classif, k = 5))
categorical_pipeline = make_pipeline(OneHotEncoder())

In [ ]:
#On re-combine le tout dans notre preprocesseur
preprocessor = make_column_transformer((numerical_pipeline, numerical_features), (categorical_pipeline, categorical_features))

## Modellisation

In [ ]:
#liste des différents modèles que l'on va essayer
knc = make_pipeline(preprocessor, KNeighborsClassifier())
linear_svc = make_pipeline(preprocessor, LinearSVC())
poly_svc = make_pipeline(preprocessor, SVC(kernel = "poly"))
rbf_svc = make_pipeline(preprocessor, SVC(kernel = "rbf"))
sigmoid_svc = make_pipeline(preprocessor, SVC(kernel = "sigmoid"))

In [ ]:
#On les places dans un dictionnaire pour pouvoir itérer dessus
dict_of_models = {'KNC' : knc, 'Linear SVC' : linear_svc, 'Poly SVC' : poly_svc, 'RBF SVC' : rbf_svc, 'Sigmoïd SVC' : sigmoid_svc}

## Procédure d'évaluation

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve

In [ ]:
#Fonction d'évaluation, renvoyant matrice de confusion, rapport de  classification et tracé des courbes d'apprentissage entre validation set et 
#train set
def evaluation(model) : 
    
    model.fit(X_train, y_train)
    ypred = model.predict(X_test)
    
    print(confusion_matrix(y_test, ypred))
    print(classification_report(y_test, ypred))
    
    N, train_score, val_score = learning_curve(model, X_train, y_train, cv = 4, scoring = 'f1', train_sizes = np.linspace(0.1, 1, 10))
    
    plt.figure(figsize = (12, 8))
    plt.plot(N, train_score.mean(axis = 1), label = 'train score')
    plt.plot(N, val_score.mean(axis = 1), label = 'validation score')
    plt.legend()

In [ ]:
#On applique la fonction à nos différents modèles
for name, model in dict_of_models.items() :
    print(name)
    evaluation(model)

On retient les modèles de Polynomial et RBF SVC qui semblent procurer les meilleurs scores sans faire d'over ou d'under fitting.

## Optimisation du RBF SVC

In [ ]:
#On va effectuer une optimisation par grille de recherche
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
#Après recherche, on obtient les hyperparamètres suivants :
hyper_params = {'svc__gamma' : ['auto'], 
                'svc__C' : [225]}

In [ ]:
#On entraine notre meilleur modèle
grid = GridSearchCV(rbf_svc, hyper_params, scoring = "f1", cv = 4)

grid.fit(X_train, y_train)

print(grid.best_params_)

y_pred = grid.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
evaluation(grid.best_estimator_)

 Précédemment, nous obtenions : 
 
 Précision : 0.85,       Recall : 0.36,       Score f1 : 0.50
 
 Après optimisiation, on parvient à améliorer le recall :
 
 Précision : 0.85,       Recall : 0.47,       Score f1 : 0.61
 

## Precision Recall Curve

Nous allons désormais essayer de créer une situation de compromis, afin de gagner un peu de recall au détriment de la précision.

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
precision, recall, threshold = precision_recall_curve(y_test, grid.best_estimator_.decision_function(X_test))

In [ ]:
#On affiche la précision et le recall pour différents seuils
plt.plot(threshold, precision[:-1], label = 'precision')
plt.plot(threshold, recall[:-1], label = 'recall')
plt.legend()
plt.show()

In [ ]:
#On crée notre modèle final de prédiction
def model_final(model, X, threshold = 0):
    return model.decision_function(X) > threshold

In [ ]:
#Après quelques essais, on place le seuil sur -0.65 pour un compromis optimal
y_pred = model_final(grid.best_estimator_, X_test, threshold = -0.65)

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, fmt = 'd')
plt.show()

La heatmap nous permet de visualiser les prédictions de notre modèle.

In [ ]:
print(classification_report(y_test, y_pred))

### On obtient une précision de 0.72 et un recall de 0.60.

### Notre objectif est atteint !